In [1]:
import btk
import os

In [2]:
path2c3d = os.path.abspath('./examples_files/DavOH6H1_1.c3d')

### Read an acquisition

In [3]:
reader = btk.btkAcquisitionFileReader()
reader.SetFilename(path2c3d)
reader.Update()
acq = reader.GetOutput()

In [4]:
type(acq)

btk.btkAcquisition

### Explore and acquisition

In [4]:
# metadata
kind = ['markers', 'emg']
metadata = {
    'first_frame': acq.GetFirstFrame(),
    'last_frame': acq.GetLastFrame(),
}

if 'markers' in kind:
    metadata.update({
        'point_rate': acq.GetPointFrequency(),
        'point_used': acq.GetPointNumber()
    })
if 'force' in kind or 'emg' in kind:
    metadata.update({
        'analog_rate': acq.GetAnalogFrequency(),
        'analog_used': acq.GetAnalogNumber()
    })
metadata

{'analog_rate': 2000.0,
 'analog_used': 38,
 'first_frame': 1,
 'last_frame': 594,
 'point_rate': 100.0,
 'point_used': 51}

In [66]:
labels = []
data = []

In [58]:
%%timeit
# analogs data
labels = []
data = []
for i in range(metadata['analog_used']):
    xi = acq.GetAnalog(i)
    data_temp = xi.GetValues()
    if data_temp.any():
        labels.append(xi.GetLabel())
        data.append(data_temp)

1.39 ms ± 4.84 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [61]:
# analogs data
labels = []
data = []
for it in btk.Iterate(acq.GetAnalogs()):
    data_temp = it.GetValues()
    if data_temp.any():
        labels.append(it.GetLabel())
        data.append(data_temp)

1.51 ms ± 27.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [68]:
%%timeit
# analogs data
analogs = {}
for it in btk.Iterate(acq.GetAnalogs()):
    data_temp = it.GetValues()
    if data_temp.any():
        analogs.update({it.GetLabel(): data_temp})

1.52 ms ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [76]:
def _iterate(acq, kind='markers'):
    out = {}
    if kind == 'markers':
        iterator = btk.Iterate(acq.GetPoints())
    if kind == 'analogs':
        iterator = btk.Iterate(acq.GetAnalogs())
    for it in iterator:
        data_temp = it.GetValues()
        if data_temp.any():
            out.update({it.GetLabel(): data_temp})
    return out

In [78]:
%%timeit
analogs = _iterate(acq, kind='analogs')

1.52 ms ± 9.65 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [72]:
# analogs data
points = {}
for it in btk.Iterate(acq.GetPoints()):
    data_temp = it.GetValues()
    if data_temp.any():
        points.update({it.GetLabel(): data_temp})

In [63]:
%%timeit
labels,data = zip(*[(it.GetLabel(), it.GetValues()) for it in btk.Iterate(acq.GetAnalogs()) if it.GetValues().any()])

1.83 ms ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
